# Analyse des Résultats ITC

Ce notebook présente une analyse interactive des résultats du benchmark ITC.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from utils.visualization import ResultsVisualizer

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# Chargement des résultats
results = pd.read_csv('../results/raw_results/individual_metrics_results.csv')
print(f"Dimensions des données: {results.shape}")
print(f"Métriques évaluées: {results['metric'].nunique()}")
print(f"Datasets utilisés: {results['dataset'].nunique()}")

results.head()

In [ ]:
# Performance moyenne par métrique
performance = results.groupby('metric').agg({
    'accuracy': ['mean', 'std'],
    'tree_depth': ['mean', 'std'],
    'training_time': ['mean', 'std']
}).round(4)

performance.columns = ['_'.join(col).strip() for col in performance.columns.values]
performance = performance.sort_values('accuracy_mean', ascending=False)

print("Top 10 des métriques par accuracy:")
performance.head(10)

In [ ]:
# Visualisation des performances
visualizer = ResultsVisualizer()
fig = visualizer.create_performance_comparison_plot(results)
plt.show()

In [ ]:
# Analyse détaillée d'ITC vs Gini
itc_vs_gini = results[results['metric'].isin(['itc', 'gini'])]

comparison = itc_vs_gini.groupby(['dataset', 'metric'])['accuracy'].mean().unstack()
comparison['improvement'] = ((comparison['itc'] - comparison['gini']) / comparison['gini']) * 100

print("Amélioration d'ITC vs Gini par dataset:")
print(comparison.sort_values('improvement', ascending=False))

## Analyse de Sensibilité des Paramètres ITC

In [ ]:
# Chargement des résultats de sensibilité
try:
    sensitivity_results = pd.read_csv('../results/raw_results/itc_parameter_sensitivity.csv')
    
    # Heatmap de performance par paramètres
    pivot_data = sensitivity_results.groupby(['alpha', 'gamma'])['accuracy'].mean().unstack()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(pivot_data, annot=True, fmt='.3f', cmap='viridis')
    plt.title('Accuracy moyenne par paramètres α et γ (β=3.5)')
    plt.tight_layout()
    plt.show()
    
except FileNotFoundError:
    print("Fichier de sensibilité non trouvé. Exécutez d'abord l'analyse de sensibilité.")